# Search

In this assessment, you'll implement a basic search engine by defining your own Python classes. A **search engine** is an algorithm that takes a query and retrieves the most relevant documents for that query. In order to identify the most relevant documents, our search engine will use **term frequency–inverse document frequency** ([tf–idf](https://en.wikipedia.org/wiki/Tf%E2%80%93idf)), an information statistic for determining the relevance of a term to each document from a corpus consisting of many documents.

The tf–idf statistic consists of two components: term frequency and inverse document frequency. **Term frequency** computes the number of times that a term appears in a **document** (such as a single Wikipedia page). If we were to use only the term frequency in determining the relevance of a term to each document, then our search result might not be helpful since most documents contain many common words such as "the" or "a". In order to downweight these common terms, the **document frequency** computes the number of times that a term appears across the **corpus** of all documents. The **tf–idf** statistic takes a term and a document and returns the term frequency divided by the document frequency.

In [136]:
%pip install -q nb_mypy pytest ipytest
%reload_ext nb_mypy
%nb_mypy mypy-options --strict


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: C:\Users\Luqman\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
Version 1.0.5


Note: you may need to restart the kernel to use updated packages.


In [137]:
import os
import math
import re

import pytest
import ipytest
ipytest.autoconfig()


def clean(token: str, pattern: re.Pattern[str] = re.compile(r"\W+")) -> str:
    """
    Returns all the characters in the token lowercased and without matches to the given pattern.

    >>> clean("Hello!")
    'hello'
    """
    return pattern.sub("", token.lower())

## Task: `Document`

Write and test a `Document` class in the code cell below that can be used to represent the text in a web page and includes methods to compute term frequency. (But not document frequency since that would require access to all the documents in the corpus.)

The `Document` class should include:

1. An initializer `__init__` takes a `str` path and initializes the document data based on the text in the specified file. Assume that the file exists, but that it could be empty. In order to implement `term_frequency` later, we'll need to precompute and save the term frequency for each term in the document in the initializer as a field by constructing a dictionary that maps each `str` term to its `float` term frequency. Term frequency is defined as *the count of the given term* divided by *the total number of words in the text*.
> Consider the term frequencies for this document containing 4 total words: "the cutest cutest dog".
>
> - "the" appears 1 time out of 4 total words, so its term frequency is 0.25.
> - "cutest" appears 2 times out of 4 total words, so its term frequency is 0.5.
> - "dog" appears 1 time out of 4 total words, so its term frequency is 0.25.

   When constructing this dictionary, call the `clean` function to lowercasing letters and ignore non-letter characters so that "corgi", "CoRgi", and "corgi!!" are all considered the same string to the search algorithm.

1. A method `term_frequency` that takes a given `str` term and returns its term frequency by looking it up in the precomputed dictionary. Remember to normalize the term before looking it up to find the corresponding match. If the term does not occur, return 0.

1. A method `get_path` that returns the `str` path of the file that this document represents.

1. A method `get_words` that returns a `set` of the unique, cleaned words in this document.

1. A method `__repr__` that returns a string representation of this document in the format `Document('{path}')` (with literal single quotes in the output) where `{path}` is the path to the document from the initializer. The `__repr__` method is called when Jupyter Notebook needs to display a `Document` as output, so we should be able to copy the string contents into a new code cell and immediately run it to create a new `Document`.

**For each of the 4 methods (excluding the initializer) in the `Document` class, write a testing function that contains at least 3 `pytest`-style assertions based on your own testing corpus**. Documentation strings are optional for testing functions.

We've provided some example corpuses in the `doggos` directory and the `small_wiki` directory. For this task, create your own testing corpus by creating a **New Folder** and adding your own text files to it.

> Be sure to exhaustively test your `Document` class before moving on: bugs in `Document` will make implementing the following `SearchEngine` task much more difficult.

In [138]:
%%ipytest

from typing import Dict

class Document:
    def __init__(self, path: str) -> None:
        # initialize fields
        self.path = path
        self.tf: Dict[str, float] = {}

        with open(path) as f:  # parse through the file
            lines = f.readlines()
            num_tokens = 0     # counter for the tokens
            for line in lines:
                tokens = line.split()
                for token in tokens:
                    token = clean(token)
                    if token in self.tf.keys():
                        self.tf[token] += 1  # update if already in self.tf
                    else:
                        self.tf[token] = 1   # initialize to 1 if not in self.tf
                    num_tokens += 1          # update num_tokens

        # iterate through the tokens in self.tf
        for token in self.tf.keys():
            # reset the value to be the term_frequency
            self.tf[token] = float(self.tf[token] / num_tokens)

    def term_frequency(self, term: str) -> float:
        '''
        Takes a given term and finds the term frequency of that term within
        the current document. No calculation needed since we preprocessed it
        within the initializer

        Arguments:
            - term: the term to find the term frequency for

        Returns:
            - the term frequency for the given term. 0 if not in the document
        '''
        term = clean(term)
        if term in self.tf.keys():
            return self.tf[term]
        return 0.0


    def get_path(self) -> str:
        '''
        Retrieves the path of the current document

        Returns:
            - the path of the document
        '''
        return self.path

    def get_words(self) -> set[str]:
        '''
        Finds the set of unique words within the current document

        Returns:
            - the set of unique, cleaned words in the current document
        '''
        return set(self.tf.keys())

    def __repr__(self) -> str:
        '''
        Returns a string representation of the current object with its path
        '''
        return ("Document('" + self.path + "')")


class TestDocument:
    euro = Document("small_wiki/Euro - Wikipedia.html")
    doc1 = Document("doggos/doc1.txt")
    doc2 = Document("doggos/my_test.txt")
    ...

    def test_term_frequency(self) -> None:
        assert self.euro.term_frequency("Euro") == pytest.approx(0.0086340569495348)
        assert self.doc1.term_frequency("dogs") == pytest.approx(1 / 5)

        # tests the case of a nonexistent word in the document
        assert self.doc2.term_frequency("nosir") == 0.0

    def test_get_words(self) -> None:
        assert set(w for w in self.euro.get_words() if len(w) == 1) == set([
            *"0123456789acefghijklmnopqrstuvxyz".lower() # All one-letter words in Euro
        ])
        assert self.doc1.get_words() == set("dogs are the greatest pets".split())

        # tests the case of cleaning and uniqueness since doc2 has repeat words
        # also tests double whitespace ("  ")
        assert self.doc2.get_words() == set("how are you doing".split())

    def test_repr(self) -> None:
        # tests the simple functionality of __repr__

        assert self.doc1.__repr__() == "Document('doggos/doc1.txt')"
        assert self.euro.__repr__() == "Document('small_wiki/Euro - Wikipedia.html')"

    def test_get_path(self) -> None:
        # tests the simple functionality of get_path

        assert self.doc1.get_path() == "doggos/doc1.txt"
        assert self.euro.get_path() == "small_wiki/Euro - Wikipedia.html"

    ...

....                                                                                         [100%]
4 passed in 0.01s


## Task: `SearchEngine`

Write and test a `SearchEngine` class in the code cell below that represents a corpus of `Document` objects and includes methods to compute the tf–idf statistic between a given query and every document in the corpus. The `SearchEngine` begins by constructing an **inverted index** that associates each term in the corpus to the list of `Document` objects that contain the term.

To iterate over all the files in a directory, call `os.listdir` to list all the file names and join the directory to the filename with `os.path.join`. The example below will print only the `.txt` files in the `doggos` directory.

In [139]:
path = "doggos"
extension = ".txt"
for filename in os.listdir(path):
    if filename.endswith(extension):
        print(os.path.join(path, filename))

doggos\doc1.txt
doggos\doc2.txt
doggos\doc3.txt
doggos\my_test.txt


The `SearchEngine` class should include:

1. An initializer that takes a `str` path to a directory such as `"small_wiki"` and a `str` file extension and constructs an inverted index from the files in the specified directory matching the given extension. By default, the extension should be `".txt"`. Assume the string represents a valid directory, and that the directory contains only valid files. Do not recreate any behavior that is already done in the `Document` class—call the `get_words()` method! Create at most one `Document` per file.

1. A method `_calculate_idf` that takes a `str` term and returns the inverse document frequency of that term. If the term is not in the corpus, return 0. Inverse document frequency is defined by calling `math.log` on the *the total number of documents in the corpus* divided by *the number of documents containing the given term*.

1. A method `__repr__` that returns a string representation of this search engine in the format `SearchEngine('{path}')` (with literal single quotes in the output) where `{path}` is the path to the document from the initializer. The `__repr__` method is called when Jupyter Notebook needs to display a `SearchEngine` as output, so we should be able to copy the string contents into a new code cell and immediately run it to create a new `SearchEngine`.

1. A method `search` that takes a `str` **query** consisting of one or more terms and returns a `list` of relevant document paths that match at least one of the cleaned terms sorted by descending tf–idf statistic. If there are no matching documents, return an empty list. See also the [Guidance for `SearchEngine.search`](#Guidance-for-SearchEngine.search).

**For each of the 3 methods (excluding the initializer) in the `SearchEngine` class, write a testing function that contains at least 3 `pytest`-style assertions based on your own testing corpus**. Documentation strings are optional for testing functions.

In [140]:
%%ipytest
from typing import Optional, List
import pprint

class SearchEngine:

    def __init__(self, path: str, file_extension: Optional[str] = ".txt") -> None:
        # i understand that i have the Optional thing in the init prototype
        # but i get a warning saying that i'm using type None instead of str
        # whenever i try to use file_extension
        if file_extension is None:
            file_extension = ".txt"

        # fields
        self.path = path                           # directory path field
        self.documents: List[Document] = []        # list of documents in the dir
        self.file_extension = file_extension       # the file_extension
        self.doc_words: Dict[str, list[str]] = {}  # words to list of docs that contain
                                                   # the word

        # iterate through the directory specified by the path
        for file_name in os.listdir(path):
            if file_name.endswith(file_extension): # check file extension
                # full_path = os.path.join(path, file_name)
                full_path = path + "/" + file_name # join the path with the
                                                   # file name

                if os.path.isfile(full_path):      # is this path a valid one?
                    doc = Document(full_path)      # yes, create a new Document obj
                    self.documents.append(doc)
                    with open(full_path) as f:     # parse through this file
                        lines = f.readlines()
                        for line in lines:
                            tokens = line.split()
                            for token in tokens:
                                token = clean(token)  # clean the token
                                if token in self.doc_words:
                                    # only add the path once
                                    if full_path not in self.doc_words[token]:
                                        self.doc_words[token].append(full_path)
                                else:
                                    word_list = [full_path]
                                    self.doc_words[token] = word_list


        # sort the doc_words dictionary by the length of the list of documents
        # each word has
        sorted_items = sorted(self.doc_words.items(), key=lambda x: len(x[1]), reverse=True)
        self.doc_words = dict(sorted_items)

    def _calculate_idf(self, term: str) -> float:
        '''
        Calculates the idf score of a single term in the corpus of documents

        Arguments:
            - term: the term we want to calculate the idf for

        Returns:
            - the idf for the term
        '''
        num = 0
        for doc in self.documents:
            if term in doc.get_words():
                num += 1
        if num != 0:
            return math.log(len(self.documents) / num)
        else:
            return 0.0

    def __repr__(self) -> str:
        '''
        Returns the string output of the SearchEngine object with the path
        and the file extension
        '''
        return "SearchEngine('" + self.path + "', '" + self.file_extension + "')"

    def search(self, query: str) -> list[str]:
        '''
        Takes a string query consisting of one or more terms and finds all the
        relevant document paths taht match at least one of the cleaned terms
        sorted by descending tf-idf statistic.

        Arguments:
            - query: the query consisting of one or more terms that we use
                     to scower the corpus of documents finding matching paths
                     that contain this query

        Returns:
            - a list of relevant document paths that match at least one of the
              cleaned terms sorted by descending tf-idf statistic. If there
              are no matching documents, we return an empty list
        '''
        words = query.split(" ")  # split the query by white space
        doc_tf_idf = {}           # initialize an empty dictionary that will be
                                  # used to find the tf_idf for the documents
                                  # from the query
        for word in words:
            word = clean(word)    # clean the word
            if word in self.doc_words:
                for file in self.doc_words[word]:
                    doc = Document(file)

                    # calculation of tf_idf
                    tf_idf = self._calculate_idf(word) * doc.term_frequency(word)

                    if file not in doc_tf_idf:  # add file only once
                        doc_tf_idf[file] = tf_idf
                    else:
                        # this case, we add the tf_idf to the already existing score
                        doc_tf_idf[file] += tf_idf


        # this line sorts the map into a list (of the keys) by descending order of
        # values which in this case, is the tf-idf score for the documents
        # matching the query
        res = [k for k, v in sorted(doc_tf_idf.items(), key=lambda item:(item[1]), reverse=True)]

        if not res:  # if the resulting list is empty, return an empty list
            return []
        return res

class TestSearchEngine:
    doggos = SearchEngine("doggos")
    small_wiki = SearchEngine("small_wiki", ".html")
    python_files = SearchEngine("tests/python", ".py")

    def test_search(self) -> None:
        assert self.doggos.search("love dogs") == ["doggos/doc3.txt", "doggos/doc1.txt"]
        # assert self.small_wiki.search("data")[:10] == [
        #     "small_wiki/Internet privacy - Wikipedia.html",
        #     "small_wiki/Machine learning - Wikipedia.html",
        #     "small_wiki/Bloomberg L.P. - Wikipedia.html",
        #     "small_wiki/Waze - Wikipedia.html",
        #     "small_wiki/Digital object identifier - Wikipedia.html",
        #     "small_wiki/Chief financial officer - Wikipedia.html",
        #     "small_wiki/UNCF - Wikipedia.html",
        #     "small_wiki/Jackson 5 Christmas Album - Wikipedia.html",
        #     "small_wiki/KING-FM - Wikipedia.html",
        #     "small_wiki/The News-Times - Wikipedia.html",
        # ]

        # tests searching for a nonexistent word in the corpus of documents
        assert self.doggos.search("luqman") == []

        # testing search but with a .py extension (different extensions test essentially)
        assert self.python_files.search("if") == ['tests/python/test1.py']

    def test_repr(self) -> None:
        expected_repr = "SearchEngine('doggos', '.txt')"
        assert repr(self.doggos) == expected_repr

    def test_calculate_idf(self) -> None:
        # tests basic calculate_idf functionality
        assert self.doggos._calculate_idf("dogs") == 0.6931471805599453

        # tests 0.0 return of nonexisteent word for calculate_idf
        assert self.doggos._calculate_idf("world") == 0.0

...                                                                                          [100%]
3 passed in 0.01s


We recommend the following iterative software development approach to implement the `search` method.

1. Write code to handle queries that contain only a single term by collecting all the documents that contain the given term, computing the tf–idf statistic for each document, and returning the list of document paths sorted by descending tf–idf statistic.

1. Write tests to ensure that your program works on single-term queries.

1. Write code to handle queries that contain more than one term by returning all the documents that match any of the terms in the query sorted by descending tf–idf statistic. The tf–idf statistic for a document that matches more than one term is defined as the sum of its constituent single-term tf–idf statistics.

1. Write tests to ensure that your program works on multi-term queries.

Here's a walkthrough of the `search` function from beginning to end. Say we have a corpus in a directory called `"doggos"` containing 3 documents with the following contents:

- `doggos/doc1.txt` with the text `Dogs are the greatest pets.`
- `doggos/doc2.txt` with the text `Cats seem pretty okay`
- `doggos/doc3.txt` with the text `I love dogs!`

The initializer should construct the following inverted index.

```python
{"dogs":     [doc1, doc3],
 "are":      [doc1],
 "the":      [doc1],
 "greatest": [doc1],
 "pets":     [doc1],
 "cats":     [doc2],
 "seem":     [doc2],
 "pretty":   [doc2],
 "okay":     [doc2],
 "i":        [doc3],
 "love":     [doc3]}
```

Searching this corpus for the multi-term query `"love dogs"` should return a list `["doggos/doc3.txt", "doggos/doc1.txt"]` by:

1. Finding all matching documents with at least one query term. `doc3` contains the word `"love"` while both `doc1` and `doc3` contain the word `"dogs"`. `doc2` does not match any of the query terms.

1. Computing the tf–idf statistic for each matching document. For each matching document, the tf–idf statistic for a multi-word query `"love dogs"` is the sum of the tf–idf statistics for `"love"` and `"dogs"` individually.

   1. For `doc1`, the sum of 0 + 0.081 = 0.081. The tf–idf statistic for `"love"` is 0 because the term is not in `doc1`.

   1. For `doc3`, the sum of 0.366 + 0.135 = 0.501.

1. Returning the matching document paths sorted by descending tf–idf statistic.

After completing your `SearchEngine`, run the following cell to search our small Wikipedia corpus for the query "data". Try some other search queries too!

In [141]:
SearchEngine("small_wiki", ".html").search("data")

<cell>1: error: Name "SearchEngine" is not defined  [name-defined]


['small_wiki/Euro - Wikipedia.html']